In [34]:
#from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.document_loaders import DirectoryLoader
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
import openai

In [20]:
import os
os.environ["OPENAI_API_KEY"]= "Your OpenAI API Key"

In [12]:
directory = 'contents'
def load_docs(directory):
    loader = DirectoryLoader(directory)
    documents = loader.load()
    return documents

documents = load_docs(directory)
print(documents)

[Document(page_content="Genpact Privileged access Management (PAN) PRODUCTION Windows end user manual\n\nBeyondTrust Privilege Access Management\n\nGenpact\n\nWindows end user Manual\n\nThis is a draft for discussion only. It represents work in progress and may contain preliminary information which is subject to change. This draft is subject to our Quality Assurance Review.\n\nDocument control\n\n0\n\n1.Introduction\n\n1.1 Purpose This document describes the windows end user login process to PAM Solution using domain credentials/OHR ID Okta which will provide access to Powerbroker Password safe portal. This also covers procedures of accessing the target servers\n\n1.2 Scope This document includes the ways to access the target systems using various managed accounts, requesting elevation to more privileged managed accounts, approving or rejecting the requested session and various other useful features of the tool for the users.\n\n1.3 Audience This document is intended for PAM Administra

In [13]:
def split_docs(documents, chunk_size=1000, chunk_overlap=50):
    text_split = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    docs = text_split.split_documents(documents)
    return docs

docs = split_docs(documents)
print(len(docs))

14


In [21]:
embeddings = OpenAIEmbeddings()
db = Chroma.from_documents(docs, embeddings)

In [22]:
query = "What is the Login Procedure for Requests Requiring Approval?"
matching_docs = db.similarity_search(query)
matching_docs[0]

Document(page_content="5. Approver can login using his/her credentials and go to Approve page & click on requests; in Pending tab\n\n6. On the basis of authenticity/validity of the request, user can Approve or Deny/Cancel the request\n\n7. If approved, the request's status will change for the user and request can be seen in Active tab a. Click on the request to view the approver, comments & open RDP session b. Open RDP Session & rest of the steps sould be same as steps 8-11 from 3.1 as mentioned above\n\n8. If not approved, the request's status will change for the user and request can be seen in All tab a. Click on the request to view the approver (rejector) and comments to reject the request.\n\n9. All user requests Rejected/Approved/Checked-In can be seen in All tab.\n\n4. FAQ", metadata={'source': 'contents\\Genpact Privileged access Managemen.txt'})

In [23]:
query1 = "Which are the different options/but given on Password safe Portal home page?"
matching_docs1 = db.similarity_search(query1)
matching_docs1[0]

Document(page_content="Question: Which are the different options/but given on Password safe Portal home page? Answer: Password Safe mortal's home page looks like: 1. Menu - Opens up the menu elaborately 2. Home - Home Page 3. Password safe - Password Safe Page i.e. Home Page for End User 4. Favourites - Lists favourites selected devices & accounts combinations (Star marked) 5. Recently Used - Lists recently used devices 6. Systems - Lists systems and devices which are not in domain (For ex: Linux) 7. Databases Lists database devices (for ex: Oracle & sql) 8. Domain Linked Accounts - Lists devices which are in domain (for ex: Windows & Netwrk) 9. Applications - Lists devices which use other application like sql Plus, Putty (as an application) 10. Cloud - Lists cloud devices 11. Accounts - Opens up the accounts available used to access devices 12. User Login - Shows user logged in and logout options 13. Settings - changes portal's page by changing color schirme 14. Requests - Redirects A

In [29]:
matching_docs2 = db.similarity_search_with_score(query,k=3)
matching_docs2

[(Document(page_content="5. Approver can login using his/her credentials and go to Approve page & click on requests; in Pending tab\n\n6. On the basis of authenticity/validity of the request, user can Approve or Deny/Cancel the request\n\n7. If approved, the request's status will change for the user and request can be seen in Active tab a. Click on the request to view the approver, comments & open RDP session b. Open RDP Session & rest of the steps sould be same as steps 8-11 from 3.1 as mentioned above\n\n8. If not approved, the request's status will change for the user and request can be seen in All tab a. Click on the request to view the approver (rejector) and comments to reject the request.\n\n9. All user requests Rejected/Approved/Checked-In can be seen in All tab.\n\n4. FAQ", metadata={'source': 'contents\\Genpact Privileged access Managemen.txt'}),
  0.27567431330680847),
 (Document(page_content='d. Open the saved session & a new screen for RDP will open up and user can perform

In [31]:
persist_directory = "chroma_db"

vectordb = Chroma.from_documents(documents=docs, embedding=embeddings, persist_directory=persist_directory)
vectordb.persist()

In [55]:
new_db = Chroma(persist_directory=persist_directory, embedding_function=embeddings)
matching_final = new_db.similarity_search_with_score(query,k=3)
matching_final

[(Document(page_content="5. Approver can login using his/her credentials and go to Approve page & click on requests; in Pending tab\n\n6. On the basis of authenticity/validity of the request, user can Approve or Deny/Cancel the request\n\n7. If approved, the request's status will change for the user and request can be seen in Active tab a. Click on the request to view the approver, comments & open RDP session b. Open RDP Session & rest of the steps sould be same as steps 8-11 from 3.1 as mentioned above\n\n8. If not approved, the request's status will change for the user and request can be seen in All tab a. Click on the request to view the approver (rejector) and comments to reject the request.\n\n9. All user requests Rejected/Approved/Checked-In can be seen in All tab.\n\n4. FAQ", metadata={'source': 'contents\\Genpact Privileged access Managemen.txt'}),
  0.27485976224377007),
 (Document(page_content='d. Open the saved session & a new screen for RDP will open up and user can perform

In [35]:
model_name = "gpt-3.5-turbo"
llm = ChatOpenAI(model_name=model_name)

In [103]:
from langchain.chains.question_answering import load_qa_chain
chain = load_qa_chain(llm, chain_type="stuff")

In [125]:
query_final = "What is Ticket System?"
matching_docs_final = new_db.similarity_search(query_final)
# Ensure 'question' key is included in the input dictionary
input_data = {'question': query_final, 'input_documents': matching_docs_final}
answer = chain.invoke(input=input_data)
answer

{'question': 'What is Ticket System?',
 'input_documents': [Document(page_content="Question: Which are the different options/buttons given on Password Safe Portal request submission page? Answer: Password Safe Portal's request submission page looks like: 1. Account - Name of the managed account used to login on which managed system 2. Start Date - Date for the request's start 3. Access Policy Window - Access policy used for the particular account 4. Start Time - Requested duration's start time. It can be immediate or scheduled for later time. 5. Requested Duration - Duration for which the request would be active. By default, it is 2 hours and can be increased but it should not expand to the next day i.e. it should not start on 7th Sep and end on 8th Sep 6. Access Request - Type of access requested 7. Reason - A valid reason why this access is needed. 8. Ticket System - Ticket system in use in the environment 9. Ticket Number - a valid ticket number 10. Submit Request - Request submissi

In [126]:
print(answer['output_text'])

A Ticket System is a system used in an environment to manage and track requests or issues. It typically involves submitting a ticket with relevant information about a request or problem, assigning it a unique ticket number, and tracking its progress until it is resolved or completed.
